In [ ]:
from dotenv import dotenv_values
import pymongo
from elasticsearch import Elasticsearch
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta
import time
from SPARQLWrapper import SPARQLWrapper, POST, JSON, CSV
from dateutil.relativedelta import relativedelta
from sentence_transformers import SentenceTransformer
import os
import subprocess

# Configuracion entorno

In [ ]:
env_config = dotenv_values("../.env")

In [ ]:
mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

In [ ]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

In [ ]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

# Pruebas de rendimiento

## Aux funcs

In [ ]:
def load_data(start_date, end_date):
    current_start_date_str = start_date.strftime('%Y-%m-%d')
    current_end_date_str = end_date.strftime('%Y-%m-%d')

    print(f"Fecha inicio: {current_start_date_str}")
    print(f"Fecha fin: {current_end_date_str}")
    print("Cargando datos en mongo")
    os.popen(f"python ../scripts/load_mongodb.py --env-file ../.env --path ../downloads dates -s {current_start_date_str} -e {current_end_date_str}").read()
    print("Cargando datos en elastic")
    #os.popen(f"python ../scripts/load_elasticsearch.py --env-file ../.env -c 20 dates -s {current_start_date_str} -e {current_end_date_str}").read()
    print("Cargando datos en graphdb")
    os.popen(f"python ../scripts/load_graphdb.py --env-file ../.env dates -s {current_start_date_str} -e {current_end_date_str}").read()
    

In [ ]:
def batch_timing_mongo(batch_querys):
    
    timer_start = time.time()
    for query in batch_querys:
        mongo_collection.find(query)
    timer_end = time.time()

    elapsed_time = timer_end - timer_start
    print(f"Tiempo de ejecución para {len(batch_querys)} querys: {elapsed_time:.5f}")
    return elapsed_time


In [ ]:
def batch_timing_graph(batch_querys):
    
    timer_start = time.time()
    for query in batch_querys:
        sparql.setReturnFormat(JSON)
        sparql.setQuery(query)
        results = sparql.query().convert()
    timer_end = time.time()

    elapsed_time = timer_end - timer_start
    print(f"Tiempo de ejecución para {len(batch_querys)} querys: {elapsed_time:.5f}")
    return elapsed_time

## Tamaño BD variable

In [ ]:
n_repeticiones = 1000
start_date_data = datetime.strptime('2023-10-01', '%Y-%m-%d')
one_month = relativedelta(months=1)

### Limpieza de datos previos y configuración BDs

In [ ]:
# clean mongo
mongo_collection.delete_many({})
mongo_collection.drop()
# clean elastic
try:
    es_client.indices.delete(index="boe")
except:
    pass
# clean graphdb
query = """
DELETE { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""
sparql.setReturnFormat(JSON)
sparql.setQuery(query)
try:
    results = sparql.query().convert()
except Exception as e:
    print('Error al limpiar graphdb') #No funciona desde python, se debe hacer manualmente
    print(e)

In [ ]:
print(os.popen("python ../scripts/setup_dbs.py --env-file ../.env --graphdb-repo-init-file ../rdf/graphdb_init.ttl --graphdb-init-query ../rdf/graphdb_init_query.txt").read())
time.sleep(10)

In [ ]:
#check number of documents in mongo
print(f"Number of documents in mongo: {mongo_collection.count_documents({})}")
#check number of documents in elastic
print(f"Number of documents in elastic: {es_client.count(index='boe')['count']}")
#check number of triples in graphdb
query_count_triples = """
    SELECT (COUNT(*) as ?count)
    WHERE { ?s ?p ?o }
"""
sparql.setQuery(query_count_triples)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(f"Number of triples in graphdb: {results['results']['bindings'][0]['count']['value']}")

### MongoDB

#### Por fecha

In [ ]:
def mongo_by_date(end_date):
    batch_querys = []
    for i in range(n_repeticiones):
        start_date = start_date_data + timedelta(days=int(random.random() * (end_date - start_date_data).days))
        end_date = start_date + timedelta(days=1)
        query_fecha = {
                    "fecha_publicacion": {
                        "$gte": start_date,
                        "$lte": end_date,
                    }
                }
        batch_querys.append(query_fecha)
    return batch_timing_mongo(batch_querys)/n_repeticiones

#### Por indentificador

In [ ]:
def mongo_by_id():
    batch_querys = []
    identificadores = mongo_collection.distinct('identificador')
    for i in range(n_repeticiones):
        query_identificador = {
                    "identificador": random.choice(identificadores)
                }
        batch_querys.append(query_identificador)
    return batch_timing_mongo(batch_querys)/n_repeticiones

#### Por materia

In [ ]:
def mongo_by_materia():
    batch_querys = []
    codigos = mongo_collection.distinct('materias.codigo')
    for i in range(n_repeticiones):
        query_codigo = {
                    "materias.codigo": random.choice(codigos)
                }
        batch_querys.append(query_codigo)
    return batch_timing_mongo(batch_querys)/n_repeticiones

### GraphDB

In [ ]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

#### Por fecha

In [ ]:
def encode_date(date):
    return f'"{date.strftime("%Y-%m-%d")}"^^xsd:date'

In [ ]:
def graph_by_date(end_date):
    batch_querys = []
    for i in range(n_repeticiones):
        date = start_date_data + timedelta(days=int(random.random() * (end_date - start_date_data).days))
        query_fecha = PREFIXES + f"""SELECT ?entradaBOE
                                WHERE {{
                                    ?entradaBOE rdf:type :EntradaBOE .
                                    ?entradaBOE :fechaPublicacion {encode_date(date)} .
                                }}
                                """  
        batch_querys.append(query_fecha)
    return batch_timing_graph(batch_querys)/n_repeticiones

#### Por indentificador

In [ ]:
def graph_by_id():
    batch_querys = []
    identificadores = mongo_collection.distinct('identificador')
    for i in range(n_repeticiones):
        query_identificador = PREFIXES + f"""SELECT ?entradaBOE
                                WHERE {{
                                    ?entradaBOE rdf:type :EntradaBOE .
                                    ?entradaBOE :identificador "{random.choice(identificadores)}" .
                                }}
                                """     
        batch_querys.append(query_identificador)
    return batch_timing_graph(batch_querys)/n_repeticiones

### Lanzamiento de pruebas

In [ ]:
times_mongo = {}
times_graph = {}
for i in range(12):
    times_mongo_tipos = {}
    times_graph_tipos = {}

    current_start_date = start_date_data + one_month * i
    current_end_date = current_start_date + one_month * (i + 1)
    
    load_data(current_start_date, current_end_date)
    
    n_docs = mongo_collection.count_documents({})
    sparql.setQuery(query_count_triples)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    n_triples = results['results']['bindings'][0]['count']['value']

    print(f"Number of documents in mongo: {n_docs}")
    print(f"Number of documents in elastic: {es_client.count(index='boe')['count']}")
    print(f"Number of triples in graphdb: {n_triples}")
    
    print('Fecha:')
    times_mongo_tipos['date'] = mongo_by_date(current_end_date)
    times_graph_tipos['date'] = graph_by_date(current_end_date)
    print('Identificador:')
    times_mongo_tipos['id'] = mongo_by_id()
    times_graph_tipos['id'] = graph_by_id()
    print('Materia:')
    times_mongo_tipos['materia'] = mongo_by_materia()
    
    times_mongo_tipos[n_docs] = times_mongo_tipos
    times_graph_tipos[n_triples] = times_graph_tipos

    print('\n')
    print(30*'=')

In [ ]:
pprint.pprint(times_mongo)

In [ ]:
pprint.pprint(times_graph)

#### Plot de resultados

In [ ]:
x = list(times_mongo.keys())
y = {k: [v['date'], v['id'], v['materia']] for k, v in times_mongo.items()}
y = list(y.values())
plt.plot(x, y)
plt.legend(['fecha', 'identificador', 'materia'])
plt.xlabel('Número de documentos')
plt.ylabel('Tiempo de ejecución (s)')
plt.show()

## Tamaño BD fijo

In [ ]:
n_querys_mongo = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
n_querys_graph = [100, 500, 1000, 5000, 10000]

### MongoDB

In [ ]:
times_mongo = {}

#### Por fecha

In [ ]:
times_dates = {}
start_date_data = datetime.strptime('2023-01-01', '%Y-%m-%d')

for n in n_querys_mongo:
    batch_querys = []
    for i in range(n):
        start_date = start_date_data + timedelta(days=int(random.random() * 365))
        end_date = start_date + timedelta(days=1)
        query_fecha = {
                    "fecha_publicacion": {
                        "$gte": start_date,
                        "$lte": end_date,
                    }
                }
        batch_querys.append(query_fecha)
    times_dates[n] = batch_timing_mongo(batch_querys)

times_mongo['fecha'] = times_dates

#### Por indentificador

In [ ]:
identificadores = mongo_collection.distinct('identificador')
times_identificadores = {}
for n in n_querys_mongo:
    batch_querys = []
    for i in range(n):
        query_identificador = {
                    "identificador": random.choice(identificadores)
                }
        batch_querys.append(query_identificador)
    times_identificadores[n] = batch_timing_mongo(batch_querys)
times_mongo['identificador'] = times_identificadores

#### Por materia

In [ ]:
codigos = mongo_collection.distinct('materias.codigo')

for n in n_querys_mongo:
    batch_querys = []
    for i in range(n):
        query_codigo = {
                    "materias.codigo": random.choice(codigos)
                }
        batch_querys.append(query_codigo)
    times_identificadores[n] = batch_timing_mongo(batch_querys)
times_mongo['codigo'] = times_identificadores

#### Plot de resultados

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
for query_type in times_mongo:
    ax.plot(n_querys_mongo, times_mongo[query_type].values(), label=query_type)
ax.set(xlabel='n_querys', ylabel='time (s)', title='Time vs num querys')
ax.grid()
ax.legend()
plt.show()

### GraphDB

In [ ]:
times_graph = {}

In [ ]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

#### Por fecha

In [ ]:
def encode_date(date):
    return f'"{date.strftime("%Y-%m-%d")}"^^xsd:date'

In [ ]:
times_dates = {}
start_date_data = datetime.strptime('2023-01-01', '%Y-%m-%d')

for n in n_querys_graph:
    batch_querys = []
    for i in range(n):
        date = start_date_data + timedelta(days=int(random.random() * 365))
        query_fecha = PREFIXES + f"""SELECT ?entradaBOE
                                WHERE {{
                                    ?entradaBOE rdf:type :EntradaBOE .
                                    ?entradaBOE :fechaPublicacion {encode_date(date)} .
                                }}
                                """  
        batch_querys.append(query_fecha)
    times_dates[n] = batch_timing_graph(batch_querys)

times_graph['fecha'] = times_dates

#### Por indentificador

In [ ]:
identificadores = mongo_collection.distinct('identificador')
times_identificadores = {}
for n in n_querys_graph:
    batch_querys = []
    for i in range(n):
        query_identificador = PREFIXES + f"""SELECT ?entradaBOE
                                WHERE {{
                                    ?entradaBOE rdf:type :EntradaBOE .
                                    ?entradaBOE :identificador "{random.choice(identificadores)}" .
                                }}
                                """     
        batch_querys.append(query_identificador)
    times_identificadores[n] = batch_timing_graph(batch_querys)
times_graph['identificador'] = times_identificadores

#### Plot de resultados

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
for query_type in times_graph:
    ax.plot(n_querys_graph, times_graph[query_type].values(), label=query_type)
ax.set(xlabel='n_querys', ylabel='time (s)', title='Time vs num querys')
ax.grid()
ax.legend()
plt.show()